In [1]:
import dolfinx
import numpy as np
import ufl
from ufl import TestFunction, TrialFunction, dx, inner
import gmsh

from petsc4py import PETSc
from mpi4py import MPI

from dolfinx import fem, mesh, io, plot, default_scalar_type
from dolfinx.io import gmshio
from dolfinx.io.gmshio import model_to_mesh
from dolfinx.fem import FunctionSpace, form, Function
from dolfinx.fem.petsc import assemble_vector, assemble_matrix, create_vector, apply_lifting, set_bc, LinearProblem

## Creating and Meshing the Domain

In [2]:
R1 = 0.5
R2 = 1

gmsh.initialize()

gmsh.model.occ.addCircle(0, 0 ,0, r=R1, tag=1)
gmsh.model.occ.addCircle(0, 0, 0, r=R2, tag=2)

gmsh.model.occ.addCurveLoop([1], tag=1)
gmsh.model.occ.addCurveLoop([2], tag=2)

gmsh.model.occ.addPlaneSurface([2,1], tag=1)

gmsh.model.occ.synchronize()

In [3]:
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.1)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.1)

gdim = 2
gmsh.model.addPhysicalGroup(gdim, [1], tag=1)
gmsh.model.mesh.generate(gdim)
# gmsh.write("annulus.msh")

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 60%] Meshing curve 2 (Circle)
Info    : Done meshing 1D (Wall 0.000286639s, CPU 0.000547s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0228565s, CPU 0.02428s)
Info    : 350 nodes 702 elements


In [4]:
model_rank = 0
domain, cell_tags, facet_tags = model_to_mesh(gmsh.model, MPI.COMM_WORLD, model_rank)
V_sc = fem.functionspace(domain, ("Lagrange", 1))
V_vec = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim, )))

## Model setup

In [5]:
# Define temporal parameters
t = 0  # Start time
T = 5  # Final time
num_steps = 500
dt = T / num_steps  # time step size

In [6]:
# Create initial condition
def initial_condition(x, a=0.4):
    r = np.sqrt(x[0]**2 + x[1]**2) - R1 # distance to inner arc/implant
    return 1 + 4*np.exp(-(a*r)**2)

def initial_condition_collagen(x, a=0.5):
    r = np.sqrt(x[0]**2 + x[1]**2) - R1 # distance to inner arc/implant
    return 1 + 0*np.exp(-(a*r)**2)

n_n = fem.Function(V_sc)
n_n.name = "Cells"
n_n.interpolate(initial_condition)

p_n = fem.Function(V_sc)
p_n.name = "Collagen"
p_n.interpolate(initial_condition_collagen)

u_n = fem.Function(V_vec)
u_n.name = "Deformation"

vel = fem.Function(V_vec)
vel.name = "Speed"

In [7]:
# Create boundary condition
def inner_arc(x):
    r = np.sqrt(x[0]**2 + x[1]**2)
    return np.isclose(r,R1)

def outer_arc(x):
    r = np.sqrt(x[0]**2 + x[1]**2)
    return np.isclose(r, R2)

fdim = domain.topology.dim - 1
inner_arc = mesh.locate_entities_boundary(domain, fdim, inner_arc)
outer_arc = mesh.locate_entities_boundary(domain, fdim, outer_arc)
boundary_facets = mesh.locate_entities_boundary(
    domain, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))

u_D = np.array([0, 0, 0], dtype=default_scalar_type)
bc_u = fem.dirichletbc(u_D, fem.locate_dofs_topological(V_vec, fdim, inner_arc), V_vec)

bc_n = fem.dirichletbc(PETSc.ScalarType(1), fem.locate_dofs_topological(V_sc, fdim, outer_arc), V_sc)
bc_p = fem.dirichletbc(PETSc.ScalarType(1), fem.locate_dofs_topological(V_sc, fdim, outer_arc), V_sc)

## Time Dependent Output

In [8]:
xdmf = io.XDMFFile(domain.comm, "IMAX.xdmf", "w")
xdmf.write_mesh(domain)

# Define solution variable, and interpolate initial solution for visualization in Paraview
nh = fem.Function(V_sc)
nh.name = "Cells"
nh.interpolate(initial_condition)
xdmf.write_function(nh, t)

ph = fem.Function(V_sc)
ph.name = "Collagen"
ph.interpolate(initial_condition_collagen)
xdmf.write_function(ph, t)

# Function with all zero entries by default
uh = fem.Function(V_vec)
uh.name = "Deformation"
# xdmf.write_function(uh, t)

In [9]:
n, v_n = ufl.TrialFunction(V_sc), ufl.TestFunction(V_sc)
p, v_p = ufl.TrialFunction(V_sc), ufl.TestFunction(V_sc)
u, v_u = ufl.TrialFunction(V_vec), ufl.TestFunction(V_vec)

T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

ds = ufl.Measure("ds", domain=domain)

In [10]:
E1 = 1.25
E2 = 1
mu1 = 1
mu2 = 1
x = ufl.SpatialCoordinate(domain)

def epsilon(u):
    return ufl.sym(ufl.grad(u))  # Equivalent to 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)

def sigma_e(u):
    return E1 * epsilon(u)*  + ufl.nabla_div(u) * ufl.Identity(len(u)) 

def sigma_v(vel):
    return mu1 * epsilon(vel) + mu2 * ufl.nabla_div(vel) * ufl.Identity(len(vel))

In [ ]:
for i in range(num_steps):
    t += dt
    if i%10 == 0:
        print(nh.x.array[100])

    # 1. Update traction force and stress, solve reaction-diffusion problem
    tau = 1
    a = 4
    k2 = 5
    N0 = 2
    hill_n = nh**k2/(N0**k2+nh**k2)
    # traction = tau*nh*ph*ufl.Identity(len(x))
    traction = tau*hill_n*ph*ufl.Identity(len(x))

    k1 = 5
    sig = sigma_e(uh) + sigma_v(vel/dt) + traction
    trsig = ufl.tr(sig)
    lin_sig = a*trsig
    hill_sig = a*ufl.real((trsig**k1)/(1+trsig**k1)) # complex numbers arises here
    
    a_n = (1 + 1/dt) * ufl.inner(n, v_n) * ufl.dx + ufl.inner(ufl.grad(n), ufl.grad(v_n)) * ufl.dx
    L_n = ufl.inner((n_n/dt), v_n) * ufl.dx + ufl.inner((1 + hill_sig), v_n) * ufl.dx

    linear_form = fem.form(L_n)
    bilinear_form = fem.form(a_n)
    A = assemble_matrix(bilinear_form, bcs=[bc_n])
    A.assemble()
    b = create_vector(linear_form)

    solver = PETSc.KSP().create(domain.comm)
    solver.setOperators(A)
    solver.setType(PETSc.KSP.Type.PREONLY)
    solver.getPC().setType(PETSc.PC.Type.LU)

    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    assemble_vector(b, linear_form)

    # Apply Dirichlet boundary condition to the vector
    apply_lifting(b, [bilinear_form], [[bc_n]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b, [bc_n])

    # Solve linear problem
    solver.solve(b, nh.x.petsc_vec)
    nh.x.scatter_forward()

    # 2. Solve collagen problem 
    d2 = 1
    f_p = nh 
    a_p = (d2 + 1/dt) * ufl.inner(p, v_p) * ufl.dx 
    L_p = ufl.inner((p_n/dt + f_p), v_p) * ufl.dx

    bilinear_form = fem.form(a_p)
    linear_form = fem.form(L_n)
    A = assemble_matrix(bilinear_form, bcs=[bc_p])
    A.assemble()
    b = create_vector(linear_form)

    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    assemble_vector(b, linear_form)

    # Apply Dirichlet boundary condition to the vector
    apply_lifting(b, [bilinear_form], [[bc_p]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b, [bc_p])

    # Solve linear problem
    solver.solve(b, ph.x.petsc_vec)
    ph.x.scatter_forward()

    # 3. Solve mechanical problem
    f_u = ufl.nabla_div(traction)
    function1 = (E1 + mu1/dt) * epsilon(u) + (E2 + mu2/dt) * ufl.nabla_div(u) * ufl.Identity(len(u)) 
    function2 = (mu1/dt) * epsilon(u_n) + (mu2/dt) * ufl.nabla_div(u_n) * ufl.Identity(len(u_n))
    
    a_u = ufl.inner(function1, epsilon(v_u)) * ufl.dx
    L_u = ufl.inner(function2, epsilon(v_u)) * ufl.dx + ufl.inner(f_u, v_u) * ufl.dx + ufl.inner(T, v_u) * ds
    problem = LinearProblem(a_u, L_u, bcs=[bc_u], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
    uh = problem.solve()

    # 4. Compute velocity at each grid point and update mesh
    ## Not sure if function spaces should be updated here??
    vel.x.array[:] = uh.x.array - u_n.x.array
    deformation_array = vel.x.array.reshape((-1, domain.geometry.dim))
    domain.geometry.x[:, :domain.geometry.dim] += deformation_array.astype(float)

    # Update solution at previous time step 
    u_n.x.array[:] = uh.x.array
    n_n.x.array[:] = nh.x.array
    p_n.x.array[:] = ph.x.array

    # Write solution to file
    nh.name = "Cells"
    ph.name = "Collagen"
    uh.name = "Deformation"
    xdmf.write_function(nh, t)
    xdmf.write_function(ph, t)
    xdmf.write_function(uh, t)
    
xdmf.close()

(5+0j)
(2.526529928182899+0j)
(1.49245848285668+0j)
